In [ ]:
import pandas as pd
import os
import numpy as np

In [ ]:
from tokenizing import *

In [ ]:
import altair as alt

In [ ]:
DATADIR = os.getenv('DATADIR')

In [ ]:
# tok = load_tokenizer_from_file(os.path.join(DATADIR, 'combined_text_tokenizer.json'))

In [ ]:
# tok.word_index

In [ ]:
# import csv


# with open(os.path.join(DATADIR, 'combined_text_word_index.csv'),'w') as f:
#     w = csv.writer(f, delimiter='\t')
#     w.writerow(["Word", "Frequency"])
#     w.writerows(tok.word_index.items())

In [ ]:
metrics = pd.read_csv(os.path.join(DATADIR, 'level_agnostic.h5plotting_metrics.csv'))

In [ ]:
metrics.shape

In [ ]:
labelled = pd.read_csv(
        os.path.join(DATADIR, 'labelled.csv.gz'),
        dtype=object,
        compression='gzip'
         )

In [ ]:
labelled.groupby('taxon_id').size().sort_values(ascending=False).plot(kind='hist', figsize=(10,5))

In [ ]:
metrics_level = pd.merge(metrics, labelled, how='left', on='taxon_id')

In [ ]:
metrics_level.shape

In [ ]:
metrics_level.columns

In [ ]:
metrics_level.head()

In [ ]:
metrics_level = metrics_level.assign(level=np.where(metrics_level.level5taxon.notnull(), 5, 0))
metrics_level.loc[metrics_level['level4taxon'].notnull() & metrics_level['level5taxon'].isnull(), 'level'] = 4
metrics_level.loc[metrics_level['level3taxon'].notnull() & metrics_level['level4taxon'].isnull(), 'level'] = 3 
metrics_level.loc[metrics_level['level2taxon'].notnull() & metrics_level['level3taxon'].isnull(), 'level'] = 2 
metrics_level.loc[metrics_level['level1taxon'].notnull() & metrics_level['level2taxon'].isnull(), 'level'] = 1 

In [ ]:
metrics_level_dupes = metrics_level.copy()

In [ ]:
#content items per taxon
metrics_level_dupes.groupby('taxon_id').size().describe()

In [ ]:
#taxons per content item
metrics_level_dupes.groupby('content_id').size().describe()

In [ ]:
# metrics_level_url = os.path.join(DATADIR, 'metrics_level.json')
# metrics_level.to_json(metrics_level_url, orient='records')

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

metrics_level_dupes.groupby('level').size().plot(kind='bar', figsize=(10,5))

In [ ]:
# alt.data_transformers.enable('default', max_rows=None)
# alt.Chart(metrics_level_dupes).mark_bar().encode(
# alt.X('level:O'),
# y='count()',
# )

In [ ]:
metrics_level.drop_duplicates(subset=['taxon_label'], inplace=True)

In [ ]:
metrics_level.shape

In [ ]:
metrics_level[metrics_level['dev_support']>=5].shape

In [ ]:
metrics_level[metrics_level['dev_support']==0].shape

In [ ]:
2139/14849

In [ ]:
chart = alt.Chart(metrics_level[metrics_level['dev_support']>0]).mark_bar().encode(
alt.X('dev_support:Q', bin=alt.BinParams(maxbins=100)),
y='count()',
).interactive()

# chart.serve()

In [ ]:
alt.Chart(metrics_level[(metrics_level['dev_support']>0)]).mark_area(opacity=0.3, interpolate='step').encode(
alt.X('dev_f1:Q', bin=alt.BinParams(maxbins=20)),
alt.Y('count()', stack=None),
alt.Color('level:O', scale=alt.Scale(scheme="accent"))

)


In [ ]:
hist =  alt.Chart(metrics_level[(metrics_level['dev_support']>0) & (metrics_level['dev_f1']>0)]).mark_area(opacity=0.8, interpolate='step').encode(
                    alt.X('dev_f1:Q', bin=alt.BinParams(maxbins=100)),
                    alt.Y('count()', stack=None),
                    alt.Color('level:O', scale=alt.Scale(scheme="accent")),
                    alt.Row('level:O')

)
hist

In [ ]:
# hist.save('level_agnostic_histograms.html')

In [ ]:
alt.Chart(metrics_level[metrics_level['level']==2]).mark_point().encode(
    x='dev_support',
    y='dev_f1',
    tooltip=['taxon_name:N', 'taxon_base_path:N', 'level:N'], 
    color='level:N'
)

In [ ]:
alt.Chart(metrics_level[metrics_level['level']==3]).mark_point().encode(
    x='dev_support',
    y='dev_f1',
    tooltip=['taxon_name:N', 'taxon_base_path:N', 'level:N'], 
    color='level:N'
)

In [ ]:
metrics_level[metrics_level['taxon_name']=='Air accidents and serious incidents']

In [ ]:
alt.Chart(metrics_level[metrics_level['level']==4]).mark_point().encode(
    x='dev_support',
    y='dev_f1',
    tooltip=['taxon_name:N', 'taxon_base_path:N', 'level:N'], 
    color='level:N'
)

In [ ]:
alt.Chart(metrics_level[metrics_level['level']==5]).mark_point().encode(
    x='dev_support',
    y='dev_f1',
    tooltip=['taxon_name:N', 'taxon_base_path:N', 'level:N', 'dev_f1'], 
    color='level:N'
)

In [ ]:
chart = alt.Chart(metrics_level[metrics_level['dev_support']>0]).mark_point().encode(
    alt.X('dev_support', title='Number of content items in dev set')
    alt.Y('dev_f1', title='Performance '
    tooltip=['taxon_name:N', 'taxon_base_path:N', 'level:N', 'dev_support:Q', 'dev_f1:Q'], 
    color='level:N'
).properties(
title='Level agnostic model, all levels shown, data=10/7/18')
chart.save('level_agnostic_results.html')

In [ ]:
chart = alt.Chart(metrics_level[metrics_level['level']>2]).mark_point().encode(
    x='dev_support',
    y='dev_f1',
    tooltip=['taxon_name:N', 'taxon_base_path:N', 'level:N'], 
    color='level:N'
).properties(
title='Level agnostic model, only deeper levels shown, data=10/7/18')
chart.save('level_agnostic_results_deeper.html')

In [ ]:
alt.Chart(metrics_level[(metrics_level['dev_f1']>=0.7) & (metrics_level['dev_support']>5)]).mark_bar().encode(
    x=alt.X('taxon_base_path:N', sort=alt.EncodingSortField(field='level', op='median', order='ascending')),
y='dev_f1:Q', 
color='level:O'
)

In [ ]:
metrics_level[metrics_level['dev_f1']>=0.8].shape

In [ ]:
metrics_level[metrics_level['dev_f1']>=0.7].shape

In [ ]:
metrics_level[metrics_level['dev_f1']>=0.7].shape

In [ ]:
alt.Chart(metrics_level[metrics_level['dev_support']>=5]).mark_bar().encode(
x='level5taxon',
y='dev_support')

In [ ]:
alt.Chart(metrics_level[metrics_level['dev_support']>=5]).mark_bar().encode(
x='level4taxon',
y='dev_f1')

In [ ]:
alt.Chart(metrics_level[metrics_level['dev_support']>=5]).mark_bar().encode(
x='level5taxon',
y='dev_f1')

In [ ]:
import json

In [ ]:
taxon_codes = pd.Series(np.load(os.path.join(DATADIR, 'levelagnostic_taxon_codes.npy')))

with open(os.path.join(DATADIR, "agnostictaxon_labels_index.json"), 'r') as f:
    labels_index = json.load(f, object_hook=lambda d: {int(k): [int(i) for i in v] if isinstance(v, list) else v for k, v in d.items()})

In [ ]:
len(taxon_codes)

In [ ]:
len(labels_index)

In [ ]:
dev = np.load(os.path.join(DATADIR, 'level_agnostic_dev_arrays.npz'))

In [ ]:
train = np.load(os.path.join(DATADIR, 'level_agnostic_train_arrays.npz'))

In [ ]:
train.files

In [ ]:
y = train['y'].any().todense()

In [ ]:
y.shape

In [ ]:
pd.DataFrame(np.sum(y, axis=0)) # frequency of taxon in training data

In [ ]:
!ls /data/2018-07-10

In [ ]:
train_results = pd.read_csv(
        os.path.join(DATADIR, 'train_results.csv.gz'),
        dtype=object,
        compression='gzip'
         )

In [ ]:
y_pred = train_results.copy()
y_pred[y_pred >= 0.5] = 1
y_pred[y_pred < 0.5] = 0

In [ ]:
true_train = pd.read_csv(
        os.path.join(DATADIR, 'true_train.csv.gz'),
        dtype=object,
        compression='gzip'
         )

In [ ]:
true_train.shape

In [ ]:
dev_results = pd.read_csv(
        os.path.join(DATADIR, 'dev_results.csv.gz'),
        dtype=object,
        compression='gzip'
         )

In [ ]:
true_dev = pd.read_csv(
        os.path.join(DATADIR, 'true_dev.csv.gz'),
        dtype=object,
        compression='gzip'
         )

In [ ]:
y_pred_dev = train_results.copy()
y_pred_dev[y_pred_dev >= 0.5] = 1
y_pred_dev[y_pred_dev < 0.5] = 0

In [ ]:
def create_df_from_array(npz_array,val):
    df = pd.DataFrame()
    df[val] = npz_array[val]
    return df

In [ ]:
df_prob = create_df_from_array(train,'content_id')
df_prob = pd.concat([df_prob, y_train], axis=1, join='inner')

In [ ]:
from sklearn.metrics import confusion_matrix, f1_score, precision_score, recall_score 
from sklearn.metrics import precision_recall_fscore_support, classification_report